## Day 49 Lecture 2 Assignment

In this assignment, we will apply mean shift clustering to a dataset containing the results of a survey on financial wellbeing.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.preprocessing import StandardScaler

This dataset contains the results of a survey on a financial wellbeing conducted by the US Consumer Finance Protection Bureau that were published in October 2017. This dataset has a large number of columns, most of which correspond to specific questions on the survey. These codebook for translating the column names to questions can be found here:

https://s3.amazonaws.com/files.consumerfinance.gov/f/documents/cfpb_nfwbs-puf-codebook.pdf

Load the dataset.

In [2]:
# answer goes here
finance = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/financial_wellbeing.csv')
print(finance.shape)
finance.head()

(6394, 217)


,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,FWB1_4,FWB1_5,FWB1_6,FWB2_1,FWB2_2,FWB2_3,FWB2_4,FSscore,FS1_1,FS1_2,FS1_3,FS1_4,FS1_5,FS1_6,FS1_7,FS2_1,FS2_2,FS2_3,SUBKNOWL1,ACT1_1,ACT1_2,FINGOALS,PROPPLAN_1,PROPPLAN_2,PROPPLAN_3,PROPPLAN_4,MANAGE1_1,MANAGE1_2,MANAGE1_3,MANAGE1_4,...,SOCSEC2,SOCSEC3,LIFEEXPECT,HHEDUC,KIDS_NoChildren,KIDS_1,KIDS_2,KIDS_3,KIDS_4,EMPLOY,EMPLOY1_1,EMPLOY1_2,EMPLOY1_3,EMPLOY1_4,EMPLOY1_5,EMPLOY1_6,EMPLOY1_7,EMPLOY1_8,EMPLOY1_9,RETIRE,MILITARY,Military_Status,agecat,generation,PPEDUC,PPETHM,PPGENDER,PPHHSIZE,PPINCIMP,PPMARIT,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,3,2,3,2,3,2,4,44,3,3,4,3,3,3,4,4,3,4,5,4,3,1,5,4,4,3,4,4,2,4,...,62,-2,-2,4,-1,0,0,0,0,8,0,0,0,0,0,0,0,1,0,1,0,5,8,1,4,1,1,1,7,3,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,3,3,4,2,2,2,3,43,3,3,3,3,4,3,2,4,3,2,5,4,3,0,3,2,2,1,4,4,1,4,...,-2,66,90,2,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,2,1,1,2,6,3,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,5,3,3,1,4,4,4,4,3,3,3,3,...,-2,68,78,3,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,3,2,1,3,6,3,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,-1,-1,-1,-1,3,3,3,3,4,4,2,4,...,-2,-1,-1,-1,-1,0,0,0,0,99,0,0,0,0,0,0,0,0,1,-2,-1,-1,3,3,2,1,1,1,8,3,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,4,3,3,1,3,3,3,3,3,3,3,3,...,-2,65,75,2,1,0,0,0,0,2,0,1,0,1,0,0,0,0,0,-2,0,5,2,4,2,3,1,5,7,1,1,2,4,0,0,1,0,4,1,4.260668


While the survey questions have the potential for interesting cluster analysis, we will stick to the "score" columns to avoid clustering in an unreasonably high-dimensional space. The columns we are interested in all have "score" in their names; identify and isolate these columns. (There should be 4 in total.)

In [3]:
# answer goes here
#filtering for score columns
score = finance.filter(like='score', axis=1)
print(score.shape)
score.head()

(6394, 4)


,FWBscore,FSscore,LMscore,KHscore
0,55,44,3,1.267
1,51,43,3,-0.570
2,49,42,3,-0.188
3,49,42,2,-1.485
4,49,42,1,-1.900


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [4]:
# answer goes here
#scaling features
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(score), columns=score.columns)
X_scale

,FWBscore,FSscore,LMscore,KHscore
0,-0.073062,-0.530906,0.653830,1.624716
1,-0.355677,-0.609920,0.653830,-0.629626
2,-0.496984,-0.688935,0.653830,-0.160841
3,-0.496984,-0.688935,-0.670399,-1.752502
4,-0.496984,-0.688935,-1.994628,-2.261785
...,...,...,...,...
6389,0.350859,-0.293863,0.653830,1.624716
6390,0.209552,0.654309,-1.994628,-1.421162
6391,0.209552,0.022194,-0.670399,-1.421162
6392,-0.708944,0.259237,-0.670399,-1.421162


Run mean shift clustering on the scores in the survey dataset using the default bandwidth. Then answer the following by printing or typing as appropriate:

- How many clusters are produced? 
- What are the cluster centers?
- How many responses are assigned to each cluster?
- Are these results reasonable? If not, what changes should we make?

In [6]:
# answer goes here
#use default bandwidth
ms = MeanShift(bandwidth=None)
score['MS_bN'] = ms.fit_predict(X_scale)

print(
    f'There are {score.MS_bN.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_bN.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
print('Cluster centers for bandwith=None')
clusters.style.background_gradient()

There are 2 clusters
The amount of values in each cluster are:
 0    6326
1      68
Name: MS_bN, dtype: int64

Cluster centers for bandwith=None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FWBscore,FSscore,LMscore,KHscore
0,57.437953,50.121439,2.814341,0.217414
1,-1.000000,-1.000000,0.250000,-1.826000


It seems that the bandwidth when set to None is too high of a bandwith because there are a disproportionate amount of data points in the 2 clusters (6326 in one and only 68 in the other). When you compare the scores, especially the FWBscore and FSscore, it shows a hugh difference in the cluster centers. A way to possibly fix the results is to set a smaller bandwidth and in turn that would create more clusters.

Try changing the appropriate parameters of the mean shift algorithm to achieve a better clustering result. Answer all of the same questions from the previous clustering step.

In [8]:
# answer goes here
#experimenting with bandwidth
ms = MeanShift(bandwidth=1)
score['MS_b1'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b1.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b1.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 41 clusters
The amount of values in each cluster are:
 0     3215
1     1353
2      441
4      229
9      132
7      130
25     119
16     117
15     100
3       85
5       74
17      63
14      54
6       39
13      30
10      25
11      24
24      19
32      18
8       14
22      13
35      12
36      12
37      11
27      11
34      10
38       6
30       5
40       5
12       5
33       4
29       4
18       3
20       2
31       2
21       2
19       2
23       1
28       1
26       1
39       1
Name: MS_b1, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FWBscore,FSscore,LMscore,KHscore
0,58.727340,50.687221,3.000000,0.369296
1,52.039337,45.219462,2.000000,-0.641718
2,49.168950,41.721461,1.000000,-1.069096
3,48.155172,44.034483,0.000000,-1.351586
4,78.600000,76.800000,2.000000,-0.552033
5,53.185185,74.888889,1.000000,-1.331037
6,32.958333,73.708333,1.000000,-1.431042
7,30.300000,64.150000,2.000000,-0.587700
8,77.416667,80.333333,1.000000,-1.471500
9,86.454545,82.636364,2.000000,0.334727


In [9]:
ms = MeanShift(bandwidth=2)
score['MS_b2'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b2.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b2.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 2 clusters
The amount of values in each cluster are:
 0    6326
1      68
Name: MS_b2, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FWBscore,FSscore,LMscore,KHscore
0,57.421547,50.105086,2.812967,0.215943
1,-1.000000,-1.000000,0.250000,-1.826000


In [10]:
ms = MeanShift(bandwidth=1.5)
score['MS_b15'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b15.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b15.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 8 clusters
The amount of values in each cluster are:
 0    4096
1    2156
5      50
4      24
2      23
7      22
6      12
3      11
Name: MS_b15, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FWBscore,FSscore,LMscore,KHscore
0,58.683315,50.565689,2.943902,0.378775
1,51.626538,45.594903,1.792619,-0.705463
2,54.500000,6.833333,0.000000,-1.958333
3,-1.000000,-1.000000,0.250000,-1.826000
4,70.000000,11.000000,2.500000,-1.599000
5,95.000000,85.000000,0.000000,-2.053000
6,80.000000,10.000000,3.000000,0.712000
7,61.000000,8.000000,1.000000,-1.485000


In [11]:
ms = MeanShift(bandwidth=1.7)
score['MS_b17'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b17.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b17.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 4 clusters
The amount of values in each cluster are:
 0    5946
1     409
3      20
2      19
Name: MS_b17, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FWBscore,FSscore,LMscore,KHscore
0,58.027393,50.184094,2.868630,0.295934
1,54.222222,11.333333,0.111111,-1.851222
2,-1.000000,-1.000000,0.250000,-1.826000
3,80.000000,10.000000,3.000000,0.712000


In [12]:
ms = MeanShift(bandwidth=0.7)
score['MS_b07'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b07.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b07.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 146 clusters
The amount of values in each cluster are:
 0      1514
1      1390
2       762
3       236
9       169
       ... 
94        1
96        1
92        1
81        1
143       1
Name: MS_b07, Length: 146, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FWBscore,FSscore,LMscore,KHscore
0,60.804478,52.710448,3.000000,0.490328
1,55.172888,46.889980,3.000000,0.086558
2,52.516129,43.838710,2.000000,-0.552395
3,49.837209,41.697674,1.000000,-1.040140
4,53.453125,51.687500,1.000000,-0.899719
5,81.978261,80.521739,3.000000,1.013630
6,49.823529,41.823529,0.000000,-1.411441
7,89.833333,72.366667,3.000000,0.539667
8,59.785714,70.535714,2.000000,-0.445679
9,63.458333,75.208333,3.000000,-0.331250


In [13]:
ms = MeanShift(bandwidth=0.9)
score['MS_b09'] = ms.fit_predict(X_scale)

#results
print(
    f'There are {score.MS_b09.nunique()} clusters\n'
    f'The amount of values in each cluster are:\n {score.MS_b09.value_counts()}\n'
    )

#cluster centers
clusters = scale.inverse_transform(ms.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

There are 59 clusters
The amount of values in each cluster are:
 0     3104
1     1339
2      395
8      163
4      138
12     115
33     109
24     109
5      103
13      90
14      83
3       70
54      70
23      51
16      37
6       32
20      30
19      28
7       24
53      24
10      21
43      20
32      19
55      19
9       18
56      17
37      12
47      11
35      11
46      11
11      10
17      10
50       9
38       8
49       8
21       7
52       7
18       7
22       6
40       5
51       5
15       5
39       4
30       4
25       4
28       3
26       3
29       2
36       2
27       2
42       2
45       1
48       1
44       1
34       1
58       1
31       1
41       1
57       1
Name: MS_b09, dtype: int64



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,FWBscore,FSscore,LMscore,KHscore
0,59.805405,51.409009,3.000000,0.405396
1,52.591133,45.098522,2.000000,-0.599559
2,48.962963,41.433862,1.000000,-1.051825
3,49.729167,43.583333,0.000000,-1.406979
4,60.923077,76.102564,2.000000,-0.926692
5,79.150000,82.750000,2.000000,-0.420500
6,31.650000,75.150000,1.000000,-1.451150
7,54.947368,78.315789,1.000000,-1.439579
8,30.555556,60.222222,2.000000,-0.477944
9,35.133333,73.533333,2.000000,-1.035800


I would say that the bandwidth for the MeanShift should be between 0.9 and 1.4 as that would illustrate the different cluster centers better and at the same not form centers around individual data points.